In [ ]:
# https://nasa-openscapes.github.io/2021-Cloud-Hackathon/tutorials/02_Data_Discovery_CMR-STAC_API.html

from pystac_client import Client  
from collections import defaultdict    
import json
import geopandas
# import geoviews as gv
from cartopy import crs
# gv.extension('bokeh', 'matplotlib')
import geopandas as gpd

In [ ]:
# find hls tiles given a point

def find_hls_tiles(point=False, band=False, limit=False, collections = ['HLSL30.v2.0', 'HLSS30.v2.0'], date_range = False):

    STAC_URL = 'https://cmr.earthdata.nasa.gov/stac'


    catalog = Client.open(f'{STAC_URL}/LPCLOUD/')



    try:
        x, y = point[0], point[1]
        # print(x,y)
    except TypeError:
        print("Point must be in the form of [lat,lon]")
        raise

    point = geopandas.points_from_xy([x],[y])
    point = point[0]

    # date_range = '2017-01-01T00:00:00Z/2018-12-31T23:59:59Z'

    # JOHN - THIS IS WHERE YOU WOULD ADD IN SEARCH PARAMETERS
    if date_range:
# ['2020-01-01:00:00:00Z', '..']
        search = catalog.search(
            collections=collections, intersects = point, datetime=date_range, limit=limit)
    else:
        search = catalog.search(
            collections=collections, intersects = point)



    # print(f'{search.matched()} Tiles Found...')


    item_collection = search.get_all_items()

    if limit:
        item_collection = item_collection[:limit]

    if band:
        links = []
        if type(band) == list:
            for i in item_collection:
                for b in band:
                    link = i.assets[b].href
                    # print(link)
                    links.append(link)
        
        else:
            for i in item_collection:
                link = i.assets[band].href
                links.append(link)
    
    else:
        links =[]
        for i in item_collection:
            # print(i.assets)
            for key in i.assets:
                if key.startswith('B'):
                    # link = i.assets[key].href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/', 's3://')
                    link = i.assets[key].href

                    # print(link)
                    links.append(link)

    return links

In [ ]:
# given a reach ID, find the nodes

import glob
import netCDF4
import os
import numpy as np


data_dir = '/home/confluence/data/mnt/input/sword'





def get_reach_nodes(data_dir, reach_id):

    all_nodes = []

    files = glob.glob(os.path.join(data_dir, '*v15.nc'))
    print(f'Searching across {len(files)} continents for nodes...')

    for i in files:

        rootgrp = netCDF4.Dataset(i, "r", format="NETCDF4")

        node_ids_indexes = np.where(rootgrp.groups['nodes'].variables['reach_id'][:].data.astype('U') == str(reach_id))

        if len(node_ids_indexes[0])!=0:
            for y in node_ids_indexes[0]:
                node_id = str(rootgrp.groups['nodes'].variables['node_id'][y].data.astype('U'))
                all_nodes.append(node_id)



            # all_nodes.extend(node_ids[0].tolist())

        rootgrp.close()

    print(f'Found {len(set(all_nodes))} nodes...')
    return list(set(all_nodes))





# get_reach_nodes(data_dir,74270100221)

In [ ]:
# given a reach ID, find the lat/lon points of all nodes



import glob
import netCDF4
import os
import numpy as np

data_dir = '/home/confluence/data/mnt/input/sword'


def get_reach_node_cords(data_dir, reach_id):

    all_nodes = []

    files = glob.glob(os.path.join(data_dir, '*v15.nc'))
    print(f'Searching across {len(files)} continents for nodes...')

    for i in files:

        rootgrp = netCDF4.Dataset(i, "r", format="NETCDF4")

        node_ids_indexes = np.where(rootgrp.groups['nodes'].variables['reach_id'][:].data.astype('U') == str(reach_id))

        if len(node_ids_indexes[0])!=0:
            for y in node_ids_indexes[0]:

                lat = str(rootgrp.groups['nodes'].variables['x'][y].data.astype('U'))
                lon = str(rootgrp.groups['nodes'].variables['y'][y].data.astype('U'))
                all_nodes.append([lat,lon])



            # all_nodes.extend(node_ids[0].tolist())

        rootgrp.close()

    print(f'Found {len(all_nodes)} nodes...')
    return all_nodes











In [ ]:
# given a reach ID, create download links for any hls tiles that intersect the nodes in the reach


def find_download_links_for_reach_tiles(data_dir, reach_id):
    node_coords = get_reach_node_cords(data_dir,reach_id)
    all_links = []
    for i in node_coords:
        print(i)
        links = find_hls_tiles(i,limit=1)
        print(links)
        all_links.extend(links)

    return list(set(all_links))

In [ ]:
from shapely.geometry import Polygon, LineString, MultiPoint, Point

def find_node_tiles(all_points, all_nodes, gdf):
    tiles_dict = {}
    for index, i in enumerate(all_points):
        point = Point(i[0],i[1])
        sentinal_tiles = gdf[gdf['geometry'].covers(point)]['Name'].tolist()
        tiles_dict[all_nodes[index]] = {'point':point, 'sentinal_tile':sentinal_tiles}
    return tiles_dict
    
        

In [ ]:
# shpfile overlap


data_dir = '/home/travis/data/10-24/mnt/input/sword'
reach_id = 23216000521

# all_links =  find_download_links_for_reach_tiles(data_dir, reach_id)
all_points = get_reach_node_cords(data_dir, reach_id);all_points[:10]

In [ ]:
all_nodes = get_reach_nodes(data_dir, reach_id);all_nodes[:10]

In [ ]:
import geopandas 
import fiona
import glob
import shapely

In [ ]:
gdf = geopandas.read_file('/home/travis/data/Sentinel-2-Shapefile-Index/sentinel_2_index_shapefile.shp');gdf.head()

In [ ]:
find_node_tiles(all_points[:10], all_nodes, gdf)

In [ ]:
from shapely.geometry import Polygon, LineString, MultiPoint, Point

point = Point('45.02741982760003','0.22797328583233897');point
point = Point('0.22797328583233897','45.02741982760003')

In [ ]:
# gdf[gdf['geometry'].covers(point)]['Name'].tolist()

In [ ]:
# search for those tiles in s3

In [5]:
from pystac_client import Client  
from shapely.geometry import Polygon, LineString, MultiPoint, Point


collections = ['HLSL30.v2.0', 'HLSS30.v2.0']

STAC_URL = 'https://cmr.earthdata.nasa.gov/stac'


catalog = Client.open(f'{STAC_URL}/LPCLOUD/')


point = Point('0.22797328583233897','45.02741982760003')
search = catalog.search(collections=collections, intersects = point, datetime='2018-01-02T00:00:00Z/2018-02-02T23:59:59Z')
# search = catalog.search(collections=collections, datetime='2018-01-02T00:00:00Z/2018-01-02T23:59:59Z')   # date_range = '2018-01-01T00:00:00Z/2018-02-01T23:59:59Z')


item_collection = search.item_collection()


links =[]
for i in item_collection:
    # print(i.assets)
    for key in i.assets:
        if key.startswith('B'):
            # link = i.assets[key].href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/', 's3://')
            link = i.assets[key].href

            # print(link)
            links.append(link)
            

In [6]:
len(links)

231

In [ ]:
# maybe we pull all tiles from the last month and do overlap?

In [ ]:
import netCDF4 as ncf

In [ ]:
fp = '/home/travis/data/10-24/mnt/input/sword/na_sword_v15.nc'
data = ncf.Dataset(fp)

In [ ]:
gdf = geopandas.read_file('/home/travis/data/Sentinel-2-Shapefile-Index/sentinel_2_index_shapefile.shp');gdf.head()

In [ ]:
# take a tile, loop through nodes, find overlap, make reaches

# dict = {'tile_num' : 
#             {reach_id: 
#                 {'node_id':('lat','lon')}}}

scene_name = 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T31TCK.2017003T104213.v2.0/HLS.L30.T31TCK.2017003T104213.v2.0.B05.tif'
tile_name = scene_name.split('/')[3];tile_name


In [ ]:
# get nodes, find all the tiles for a reach